### Neural Net Math Project Notebook

Steps:
1. read data 
2. normalize data 
3. split the data, x, y 
4. make data as #feauture and #samples
5. split the data to training and testing
6. initialize parameters & hyperparameters
7. running the program

In [1]:
# import libraries and packages
import numpy as np
import pandas as pd


In [ ]:
# read the data
df_data = pd.read_csv("data/user_fake_authentic_2class.csv")
data_x = df_data.iloc[:,:-1]
data_y = df_data.iloc[:,-1:]